In [ ]:
# PG53601
# Afonso Xavier Cardoso Marques
# Mestrado em Engenharia Informática

In [ ]:
# ---------------------------------------------------------------------------------------------------------------------------------------
# ------------------------------------------------ SAT solvers - Exercício de avaliação -------------------------------------------------
# ---------------------------------------------------------------------------------------------------------------------------------------

In [29]:
"""
Pretende-se colorir uma sequência de 9 azulejos justapostos com uma paleta de 4 cores: azul, verde, cinzento e preto. A coloração dos azulejos tem que ser feita de acordo com as
seguintes regras:

    1. Cada azulejo tem uma única cor.
    2. Os azulejos das extremidades têm que ter a mesma cor.
    3. Todas as cores têm que ser usadas.
    4. O azulejo do meio só pode ser pintado de cinzento ou preto.
    5. O penúltimo azulejo da sequência tem que ser preto, excepto se o último estiver pintado de preto.
    6. Azulejos seguidos não podem ser pintados da mesma cor.
    7. Caso o primeiro azulejo seja verde e o terceiro não seja preto, o segundo terá que ser cinzento ou preto.
    8. Os azulejos na terceira e na sétima posição só podem ser pintados de azul ou de verde.

Pretende-se codificar este problema em lógica proposicional e usar o Z3 para responder a algumas questões.
"""

# Instalar z3 se necessário
!pip install z3-solver

In [30]:
"""
-----------------------------------------------------------------------------------------
EX1) Codifique o problema num solver e comprove que o conjunto de fórmulas é consistente
-----------------------------------------------------------------------------------------
"""

from z3 import *

cores = ["Azul", "Verde", "Cinzento", "Preto"]
azulejos = [1, 2, 3, 4, 5, 6, 7, 8, 9]

# Xca -> onde 'c' é uma cor e 'a' é um azulejo
x = {}
for c in cores:
    x[c] = {}
    for a in azulejos:
        x[c][a] = Bool("%s,%s" % (c, a))

s = Solver()


# Regras para o SAT Solver
# Cada azulejo tem uma única cor
for a in azulejos:
    s.add(Or(x["Azul"][a], x["Verde"][a], x["Cinzento"][a], x["Preto"][a]))
    s.add(Implies(x["Azul"][a], And(Not(x["Verde"][a]), Not(x["Cinzento"][a]), Not(x["Preto"][a]))))
    s.add(Implies(x["Verde"][a], And(Not(x["Azul"][a]), Not(x["Cinzento"][a]), Not(x["Preto"][a]))))
    s.add(Implies(x["Preto"][a], And(Not(x["Verde"][a]), Not(x["Cinzento"][a]), Not(x["Azul"][a]))))
    s.add(Implies(x["Cinzento"][a], And(Not(x["Verde"][a]), Not(x["Azul"][a]), Not(x["Preto"][a]))))

# Azulejos das extremidades têm que ter a mesma cor
for c in cores:
    s.add(Implies(x[c][1], x[c][9]))
    s.add(Implies(x[c][9], x[c][1]))

# Todas as cores têm que ser usadas
for c in cores:
    s.add(Or([x[c][a] for a in azulejos]))

# O azulejo do meio só pode ser cinzento ou preto
s.add(Or(x["Cinzento"][5], x["Preto"][5]))

# O penúltimo azulejo tem que ser preto, exceto se o último estiver pintado de preto
s.add(Or(And(x["Preto"][8], Not(x["Preto"][9])), x["Preto"][9]))

# Azulejos seguidos não podem ser pintados da mesma cor
for a in range(1, 9):
    for c in cores:
        s.add(Implies(x[c][a], Not(x[c][a + 1])))

# Se o primeiro azulejo é verde e o terceiro não é preto, o segundo deve ser cinzento ou preto
s.add(Implies(And(x["Verde"][1], Not(x["Preto"][3])), Or(x["Cinzento"][2], x["Preto"][2])))

# Os azulejos na terceira e na sétima posição só podem ser azul ou verde
s.add(Or(x["Azul"][3], x["Verde"][3]))
s.add(Or(x["Azul"][7], x["Verde"][7]))


# Verificar se as regras são satisfazíveis
if s.check() == sat:
    print("O conjunto de regras é consistente.")

else:
    print("O conjunto de regras é inconsistente.")



O conjunto de regras é consistente.


In [31]:
"""
-----------------------------------------------------------------------------------------
EX2) Com a ajuda do solver, pronuncie-se quanto à veracidade das seguintes afirmaçõess:
        (a) Se o primeiro azulejo for preto, o penúltimo azulejo tem que ser azul.
        (b) Algum dos dois últimos azulejos tem que ser preto.
        (c) Algum dos três primeiros azulejos poderá ser azul.
-----------------------------------------------------------------------------------------
"""

from z3 import *

# ---a)---
print("a) Se o primeiro azulejo for preto, o penúltimo azulejo tem que ser azul.")

re14 = Implies(x["Preto"][1], x["Azul"][8])
s.add(re14)


if s.check() == sat:
        print("É verdade!")
        print("\n")
else:
        print("É falso!")
        print("\n")

# ---b)---
print("b) Algum dos dois últimos azulejos tem que ser preto.")

re15 = Or(x["Preto"][8], x["Preto"][9])
s.add(re15)

if s.check() == sat:
        print("É verdade!")
        print("\n")
else:
        print("É falso!")
        print("\n")

# ---c)---
print("c) Algum dos três primeiros azulejos poderá ser azul.")

re16 = Or(x["Azul"][1], x["Azul"][2], x["Azul"][3])
s.add(re16)


if s.check() == sat:
        print("É verdade!")
        print("\n")
else:
        print("É falso!")
        print("\n")



a) Se o primeiro azulejo for preto, o penúltimo azulejo tem que ser azul.
É verdade!


b) Algum dos dois últimos azulejos tem que ser preto.
É verdade!


c) Algum dos três primeiros azulejos poderá ser azul.
É verdade!




In [32]:
"""
-----------------------------------------------------------------------------------------
EX3) Acrescente código ao programa de modo a saber todas as colorações que são
     possíveis fazer aos azulejos. Quantas são?
-----------------------------------------------------------------------------------------
"""

from z3 import *

s.push()

num_solucoes = 0

while s.check() == sat:
    m = s.model()

    combinacao = ""
    print("Comb%d" % (num_solucoes+1))
    combinacao += " | "

    for a in azulejos:
        for c in cores:
            if is_true(m[x[c][a]]):
                #print("Azulejo %d é %s" % (a,c))
                combinacao += a.__str__()
                combinacao += ". "
                combinacao += c
                combinacao += " | "


    print(combinacao)
    print("-----------------------------------------------------------------------------------------------------------")
    num_solucoes += 1

    restricao = []
    for a in azulejos:
        for c in cores:
            if is_true(m[x[c][a]]):
                restricao.append(Not(x[c][a]))
    s.add(Or(restricao))


else:
    if num_solucoes < 0:
        print("Não tem solução.")
    else:
        print()
        print("Encontradas %d combinações para os azulejos" % (num_solucoes))

s.pop()



Comb1
 | 1. Azul | 2. Cinzento | 3. Verde | 4. Azul | 5. Cinzento | 6. Verde | 7. Azul | 8. Preto | 9. Azul | 
-----------------------------------------------------------------------------------------------------------
Comb2
 | 1. Azul | 2. Cinzento | 3. Verde | 4. Azul | 5. Preto | 6. Verde | 7. Azul | 8. Preto | 9. Azul | 
-----------------------------------------------------------------------------------------------------------
Comb3
 | 1. Azul | 2. Cinzento | 3. Verde | 4. Azul | 5. Preto | 6. Azul | 7. Verde | 8. Preto | 9. Azul | 
-----------------------------------------------------------------------------------------------------------
Comb4
 | 1. Azul | 2. Cinzento | 3. Verde | 4. Cinzento | 5. Preto | 6. Azul | 7. Verde | 8. Preto | 9. Azul | 
-----------------------------------------------------------------------------------------------------------
Comb5
 | 1. Azul | 2. Cinzento | 3. Verde | 4. Preto | 5. Cinzento | 6. Azul | 7. Verde | 8. Preto | 9. Azul | 
-----------------